In [1]:
import os, sys, math, datetime
import pathlib
from pathlib import Path
import numpy as np
import pandas as pd
import random
from matplotlib import pyplot as plt
import PIL
import PIL.Image
import seaborn as sns

import tensorflow as tf
# import tensorflow_datasets as tfds
from tensorflow import keras
# from tensorflow.keras.layers import Input, Dense, Flatten, Conv2D,DepthwiseConv2D, MaxPooling2D, AvgPool2D, GlobalAveragePooling2D, BatchNormalization, Concatenate
# from tensorflow.keras.layers import ReLU
# from tensorflow.keras.models import Model
# from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler
 
# Import the necessary MLTK APIs
from mltk.core import view_model, summarize_model, profile_model

# import workbench.config.config
from workbench.config.config import initialize
from workbench.utils.utils import create_filepaths, get_file_size
from workbench.utils.utils import parse_model_name, parse_mltk_model_summary
from workbench.data.data import get_lemon_quality_dataset
from workbench.tflite_profiling import get_peak_memory_df, get_tensor_details_df
from workbench.tensorflow import get_layer_details_df

#from dotenv import load_dotenv
import wandb
from wandb.keras import WandbMetricsLogger, WandbModelCheckpoint
#import deeplake

%load_ext autoreload



In [2]:
%reload_ext autoreload
%autoreload

In [3]:
models_dir = initialize()

In [4]:
global model_name
#model_name = "efficientNetB0_1_96_c3_o3_keras"
#model_name = "shufflenetv2tinys_0.2_96_c3_o3_f4l1024"
model_name = "mobilenetv1_0.25_96_c3_o2_l5"

#model_name = "mobilenetv2_0.35_96_c3_o3_l5"
#model_name = "mobilenetv2_0.35_96_c3_o3_keras"
#model_name = "shufflenetv2tiny_0.1_96_c3_o3_f4l1024"
#model_name = "MobilenetV3small_1_96_c3_o3_keras"#, "MobilenetV3large_1_224_c3_o3_keras"# ,


In [5]:
base_model_name, alpha, resolution, channels, classes, variation = model_name.split("_")
alpha = float(alpha)
resolution = int(resolution)
classes = int(classes.strip("o"))
channels = int(channels.strip("c"))
PROJECT = "model_DB"

# just needed for INT 8 quantization:
BATCH_SIZE = 32
shuffle_seed = 1

In [6]:
models_path, models_summary_path, models_image_path, models_layer_df_path, models_tf_path, models_tflite_path, models_tflite_opt_path = create_filepaths(model_name)

i:\tinyml\tiny_cnn\models


In [7]:
model = keras.models.load_model(models_tf_path)

In [8]:
# view_model(model, tflite=True, build=True)

# Parse model parameters from MLTK summary

In [9]:
total_params, trainable_params, non_trainable_params, MACs, FLOPs = parse_mltk_model_summary(models_summary_path)        

In [10]:
mltk_model_stats = {}

In [11]:
mltk_model_stats["MACs"] = MACs
mltk_model_stats["FLOPs"] = FLOPs
mltk_model_stats["total_params"] = total_params
mltk_model_stats["trainable_params"] = trainable_params
mltk_model_stats["non_trainable_params"] = non_trainable_params

# Creating tflite Models

In [12]:
# Convert the model to the TensorFlow Lite format without quantization
converter = tf.lite.TFLiteConverter.from_keras_model(model)
#converter = tf.lite.TFLiteConverter.from_saved_model(models_path)
tflite_model = converter.convert()


INFO:tensorflow:Assets written to: C:\Users\Susanne\AppData\Local\Temp\tmppp59rxr7\assets


INFO:tensorflow:Assets written to: C:\Users\Susanne\AppData\Local\Temp\tmppp59rxr7\assets


In [13]:
# Save the model.
with open(models_tflite_path, "wb") as f:
    f.write(tflite_model)

In [14]:
mltk_model_stats["model_size_kb"] = get_file_size(models_tf_path)
mltk_model_stats["tflite_model_size_kb"] = get_file_size(models_tflite_path)

File size in bytes is 1150560
File size in kilobytes is 1123.59375
File size in bytes is 854536
File size in kilobytes is 834.5078125


# tflite with INT8 quantization

In [15]:
dataset_path = Path.cwd().joinpath("datasets", "lemon_dataset")
dataset_path.exists()

True

In [16]:
train_ds, val_ds, test_ds, labels = get_lemon_quality_dataset(dataset_path, resolution, resolution, BATCH_SIZE, channels)

Color mode: rgb
Preparing training dataset...
Found 2021 files belonging to 3 classes.
Preparing validation dataset...
Found 252 files belonging to 3 classes.
Preparing test dataset...
Found 255 files belonging to 3 classes.
Class names: ['bad_quality', 'empty_background', 'good_quality']
Train: (TensorSpec(shape=(None, 96, 96, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))
Normalize: True


In [17]:
repr_ds = test_ds.unbatch()

def representative_data_gen():
  for i_value, o_value in repr_ds.batch(1).take(48):
    yield [i_value]
    
converter_opt = tf.lite.TFLiteConverter.from_keras_model(model)

# set the optimization flag
converter_opt.optimizations = [tf.lite.Optimize.DEFAULT]
# enforce integer only quantization
converter_opt.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter_opt.inference_input_type = tf.uint8
converter_opt.inference_output_type = tf.uint8

# provide a representative dataset for quantization
converter_opt.representative_dataset = representative_data_gen

tflite_model_opt = converter_opt.convert()

# Save the model.
with open(models_tflite_opt_path, 'wb') as f:
  f.write(tflite_model_opt)
models_tflite_opt_path

INFO:tensorflow:Assets written to: C:\Users\Susanne\AppData\Local\Temp\tmpyedgidsi\assets


INFO:tensorflow:Assets written to: C:\Users\Susanne\AppData\Local\Temp\tmpyedgidsi\assets
d:\Miniconda\envs\tiny_cnn_6\lib\site-packages\tensorflow\lite\python\convert.py:766: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "


WindowsPath('i:/tinyml/tiny_cnn/models/mobilenetv1_0.25_96_c3_o2_l5/mobilenetv1_0.25_96_c3_o2_l5_INT8.tflite')

In [18]:
mltk_model_stats["tflite_model_INT8_size_kb"] = get_file_size(models_tflite_opt_path)

File size in bytes is 301232
File size in kilobytes is 294.171875


In [19]:
mltk_model_stats["tflite_model_size_kb"] = get_file_size(models_tflite_path)

File size in bytes is 854536
File size in kilobytes is 834.5078125


# Profiling tflite models with MLTK profiler

In [20]:
mltk_model_stats["architecture"] = model_name
mltk_model_stats["alpha"] = alpha
mltk_model_stats["img_res"] = resolution
mltk_model_stats["classes"] = classes
mltk_model_stats["channels"] = channels
mltk_model_stats["variation"] = variation

In [21]:
# MLTK profile model reads the mode from a path!

profiling_results = profile_model(str(models_tflite_path), accelerator=None, build=None)

Profiling model in simulator ...
Using Tensorflow-Lite Micro version: b13b48c (2022-06-08)
Searching for optimal runtime memory size ...
Determined optimal runtime memory size to be 324864


In [22]:
mltk_model_stats["opt_RAM_mltk"] = int(input("Please copy the optimal runtime memory size from above: \n"))

In [23]:
profiling_results.get_summary(exclude_null=False, full_summary=True)
model_profile = profiling_results.to_dict()

In [24]:
mltk_model_stats["flash_model_size_b_mltk"] = model_profile["summary"]["tflite_size"]
mltk_model_stats["RAM_runtime_memory_size_b_mltk"] = model_profile["summary"]["runtime_memory_size"]
mltk_model_stats["OPS_mltk"] = model_profile["summary"]["ops"]
mltk_model_stats["macs_mltk"] = model_profile["summary"]["macs"]
mltk_model_stats["n_unsupported_layers_mltk"] = model_profile["summary"]["n_unsupported_layers"]
mltk_model_stats["energy_mltk"] = model_profile["summary"]["energy"]

In [25]:
layers_df = pd.DataFrame.from_dict(model_profile["layers"])
layers_df["name"] = layers_df["index"].map(str) + ": " + layers_df["opcode"]
layers_df.insert(2,"name" , layers_df.pop("name"))
layers_df

,index,opcode,name,ops,macs,cpu_cycles,energy,input_shape,output_shape,options
0,0,conv_2d,0: conv_2d,1050624,497664,4.600492e+06,8.359018e-04,"1x96x96x3,8x3x3x3,8",1x48x48x8,Padding:same stride:2x2 activation:relu
1,1,depthwise_conv_2d,1: depthwise_conv_2d,387072,165888,2.522040e+06,4.679793e-04,"1x48x48x8,1x3x3x8,8",1x48x48x8,Multiplier:1 padding:same stride:1x1 activatio...
2,2,conv_2d,2: conv_2d,700416,294912,3.560665e+06,6.482228e-04,"1x48x48x8,16x1x1x8,16",1x48x48x16,Padding:valid stride:1x1 activation:relu
3,3,depthwise_conv_2d,3: depthwise_conv_2d,193536,82944,1.036370e+06,2.085251e-04,"1x48x48x16,1x3x3x16,16",1x24x24x16,Multiplier:1 padding:same stride:2x2 activatio...
4,4,conv_2d,4: conv_2d,645120,294912,1.691265e+06,2.639363e-04,"1x24x24x16,32x1x1x16,32",1x24x24x32,Padding:valid stride:1x1 activation:relu
5,5,depthwise_conv_2d,5: depthwise_conv_2d,387072,165888,2.052064e+06,3.942758e-04,"1x24x24x32,1x3x3x32,32",1x24x24x32,Multiplier:1 padding:same stride:1x1 activatio...
6,6,conv_2d,6: conv_2d,1234944,589824,2.210628e+06,3.351090e-04,"1x24x24x32,32x1x1x32,32",1x24x24x32,Padding:valid stride:1x1 activation:relu
7,7,depthwise_conv_2d,7: depthwise_conv_2d,96768,41472,5.061695e+05,9.911860e-05,"1x24x24x32,1x3x3x32,32",1x12x12x32,Multiplier:1 padding:same stride:2x2 activatio...
8,8,conv_2d,8: conv_2d,617472,294912,1.101254e+06,1.183100e-04,"1x12x12x32,64x1x1x32,64",1x12x12x64,Padding:valid stride:1x1 activation:relu
9,9,depthwise_conv_2d,9: depthwise_conv_2d,193536,82944,9.851138e+05,1.941313e-04,"1x12x12x64,1x3x3x64,64",1x12x12x64,Multiplier:1 padding:same stride:1x1 activatio...


In [26]:
layers_df_path = pathlib.Path.joinpath(models_dir, "layer_df_mltk.pkl")
layers_df.to_pickle(layers_df_path)

In [27]:
mltk_model_stats

{'MACs': '7.490 M',
 'FLOPs': '15.674 M',
 'total_params': 221794,
 'trainable_params': 216322,
 'non_trainable_params': 5472,
 'model_size_kb': 1123.59375,
 'tflite_model_size_kb': 834.5078125,
 'tflite_model_INT8_size_kb': 294.171875,
 'arhcitecture': 'mobilenetv1_0.25_96_c3_o2_l5',
 'alpha': 0.25,
 'img_res': 96,
 'classes': 2,
 'channels': 3,
 'variation': 'l5',
 'opt_RAM_mltk': 324864,
 'flash_model_size_b_mltk': 854536,
 'RAM_runtime_memory_size_b_mltk': 322644,
 'OPS_mltk': 15673996,
 'macs_mltk': 7489664,
 'n_unsupported_layers_mltk': 0,
 'energy_mltk': 0.005974172272239287}

In [28]:
# %%capture optimal_runtime_INT
profiling_results_INT8 = profile_model(str(models_tflite_opt_path), accelerator=None, build=False)

Profiling model in simulator ...
Using Tensorflow-Lite Micro version: b13b48c (2022-06-08)
Searching for optimal runtime memory size ...
Determined optimal runtime memory size to be 104576


In [29]:
mltk_model_stats["opt_RAM_INT8_mltk"] = int(input("Please copy the optimal runtime memory size from above: \n"))

In [30]:
# from contextlib import redirect_stdout

# with open("runtime.txt", "w", encoding='utf-8') as f:
#     with redirect_stdout(f):
#         print(profile_model(str(models_tflite_opt_path), accelerator=None, build=False))

In [31]:
profiling_results_INT8.get_summary()

OrderedDict([('name', 'mobilenetv1_0.25_96_c3_o2_l5_INT8'),
             ('accelerator', 'None'),
             ('input_shape', '1x96x96x3'),
             ('input_dtype', 'uint8'),
             ('output_shape', '1x2'),
             ('output_dtype', 'uint8'),
             ('tflite_size', 301232),
             ('runtime_memory_size', 101956),
             ('ops', 15784596),
             ('macs', 7489664),
             ('n_layers', 36),
             ('n_unsupported_layers', 0),
             ('cpu_cycles', 38318275.24157715),
             ('cpu_utilization', 0.0),
             ('cpu_clock_rate', 78000000),
             ('energy', 0.006173433389285777),
             ('j_per_op', 3.9110493479122156e-10),
             ('j_per_mac', 8.242603926271962e-10)])

In [32]:
model_profile_INT8 = profiling_results_INT8.to_dict()


In [33]:
mltk_model_stats["flash_model_size_b__INT8_mltk"] = model_profile_INT8["summary"]["tflite_size"]
mltk_model_stats["RAM_runtime_memory_size_b_INT8_mltk"] = model_profile_INT8["summary"]["runtime_memory_size"]
mltk_model_stats["OPS_INT8_mltk"] = model_profile_INT8["summary"]["ops"]
mltk_model_stats["macs_INT8_mltk"] = model_profile_INT8["summary"]["macs"]
mltk_model_stats["n_unsupported_layers_INT8_mltk"] = model_profile_INT8["summary"]["n_unsupported_layers"]
mltk_model_stats["energy_INT8_mltk"] = model_profile_INT8["summary"]["energy"]

In [34]:
mltk_model_stats

{'MACs': '7.490 M',
 'FLOPs': '15.674 M',
 'total_params': 221794,
 'trainable_params': 216322,
 'non_trainable_params': 5472,
 'model_size_kb': 1123.59375,
 'tflite_model_size_kb': 834.5078125,
 'tflite_model_INT8_size_kb': 294.171875,
 'arhcitecture': 'mobilenetv1_0.25_96_c3_o2_l5',
 'alpha': 0.25,
 'img_res': 96,
 'classes': 2,
 'channels': 3,
 'variation': 'l5',
 'opt_RAM_mltk': 324864,
 'flash_model_size_b_mltk': 854536,
 'RAM_runtime_memory_size_b_mltk': 322644,
 'OPS_mltk': 15673996,
 'macs_mltk': 7489664,
 'n_unsupported_layers_mltk': 0,
 'energy_mltk': 0.005974172272239287,
 'opt_RAM_INT8_mltk': 104576,
 'flash_model_size_b__INT8_mltk': 301232,
 'RAM_runtime_memory_size_b_INT8_mltk': 101956,
 'OPS_INT8_mltk': 15784596,
 'macs_INT8_mltk': 7489664,
 'n_unsupported_layers_INT8_mltk': 0,
 'energy_INT8_mltk': 0.006173433389285777}

In [35]:
layers_INT8_df = pd.DataFrame.from_dict(model_profile_INT8["layers"])
layers_INT8_df["name"] = layers_INT8_df["index"].map(str) + ": " + layers_INT8_df["opcode"]
layers_INT8_df.insert(2,"name" , layers_INT8_df.pop("name"))
layers_INT8_df

,index,opcode,name,ops,macs,cpu_cycles,energy,input_shape,output_shape,options
0,0,quantize,0: quantize,110592,0,9.957829e+05,1.988387e-04,1x96x96x3,1x96x96x3,Type=none
1,1,conv_2d,1: conv_2d,1050624,497664,4.600492e+06,8.359018e-04,"1x96x96x3,8x3x3x3,8",1x48x48x8,Padding:same stride:2x2 activation:relu
2,2,depthwise_conv_2d,2: depthwise_conv_2d,387072,165888,2.522040e+06,4.679793e-04,"1x48x48x8,1x3x3x8,8",1x48x48x8,Multiplier:1 padding:same stride:1x1 activatio...
3,3,conv_2d,3: conv_2d,700416,294912,3.560665e+06,6.482228e-04,"1x48x48x8,16x1x1x8,16",1x48x48x16,Padding:valid stride:1x1 activation:relu
4,4,depthwise_conv_2d,4: depthwise_conv_2d,193536,82944,1.036370e+06,2.085251e-04,"1x48x48x16,1x3x3x16,16",1x24x24x16,Multiplier:1 padding:same stride:2x2 activatio...
5,5,conv_2d,5: conv_2d,645120,294912,1.691265e+06,2.639363e-04,"1x24x24x16,32x1x1x16,32",1x24x24x32,Padding:valid stride:1x1 activation:relu
6,6,depthwise_conv_2d,6: depthwise_conv_2d,387072,165888,2.052064e+06,3.942758e-04,"1x24x24x32,1x3x3x32,32",1x24x24x32,Multiplier:1 padding:same stride:1x1 activatio...
7,7,conv_2d,7: conv_2d,1234944,589824,2.210628e+06,3.351090e-04,"1x24x24x32,32x1x1x32,32",1x24x24x32,Padding:valid stride:1x1 activation:relu
8,8,depthwise_conv_2d,8: depthwise_conv_2d,96768,41472,5.061695e+05,9.911860e-05,"1x24x24x32,1x3x3x32,32",1x12x12x32,Multiplier:1 padding:same stride:2x2 activatio...
9,9,conv_2d,9: conv_2d,617472,294912,1.101254e+06,1.183100e-04,"1x12x12x32,64x1x1x32,64",1x12x12x64,Padding:valid stride:1x1 activation:relu


In [36]:
layers_df_INT8_path = pathlib.Path.joinpath(models_dir, "layer_df_INT8_mltk.pkl")
layers_INT8_df.to_pickle(layers_df_INT8_path)

In [37]:
#layers_INT8_df.info()

# Getting peak memory infos

In [38]:
models_peak_memory_path = models_dir.joinpath(model_name, f"{model_name}_peak_memory")

In [39]:
! python i:\tinyml\tflite-tools\tflite_tools.py -i $models_tflite_opt_path --csv $models_peak_memory_path
#! explorer $models_peak_memory_path

Writing model analysis to i:\tinyml\tiny_cnn\models\mobilenetv1_0.25_96_c3_o2_l5\mobilenetv1_0.25_96_c3_o2_l5_peak_memory in CSV format


In [40]:
model_peak_memory_schedule = models_dir.joinpath(model_name, f"{model_name}_peak_memory", "execution_schedule_info.csv")
peak_memory_tensor_details = models_dir.joinpath(model_name, f"{model_name}_peak_memory", "tensor_details.csv")

In [41]:
peak_memory_df = get_peak_memory_df(model_peak_memory_schedule)
peak_memory_df.head(10)

Reading in i:\tinyml\tiny_cnn\models\mobilenetv1_0.25_96_c3_o2_l5\mobilenetv1_0.25_96_c3_o2_l5_peak_memory\execution_schedule_info.csv
Cleaning up the dataframe.


,name,tensor_IDs,RAM_b,operator,index
0,tfl.quantize,"(0, 61)",55296,tfl.quantize,0
1,conv2d,"(61, 62)",46080,mobilenetv1/re_lu/Relu;mobilenetv1/batch_norma...,1
2,depthwise_conv2d,"(62, 63)",36864,mobilenetv1/re_lu_1/Relu;mobilenetv1/batch_nor...,2
3,conv2d_1,"(63, 64)",55296,mobilenetv1/re_lu_2/Relu;mobilenetv1/batch_nor...,3
4,depthwise_conv2d_1,"(64, 65)",46080,mobilenetv1/re_lu_3/Relu;mobilenetv1/batch_nor...,4
5,conv2d_2,"(65, 66)",27648,mobilenetv1/re_lu_4/Relu;mobilenetv1/batch_nor...,5
6,batch_normalization_7,"(66, 67)",36864,mobilenetv1/re_lu_5/Relu;mobilenetv1/batch_nor...,6
7,conv2d_3,"(67, 68)",36864,mobilenetv1/re_lu_6/Relu;mobilenetv1/batch_nor...,7
8,depthwise_conv2d_3,"(68, 69)",23040,mobilenetv1/re_lu_7/Relu;mobilenetv1/batch_nor...,8
9,conv2d_4,"(69, 70)",13824,mobilenetv1/re_lu_8/Relu;mobilenetv1/batch_nor...,9


In [42]:
peak_memory = peak_memory_df["RAM_b"].max()
peak_memory

55296

In [43]:
mltk_model_stats["peak_memory_b"] = peak_memory

In [44]:
peak_memory_df["RAM_b"].argmax()

0

In [45]:
tensor_df = get_tensor_details_df(peak_memory_tensor_details)
tensor_df

Reading in i:\tinyml\tiny_cnn\models\mobilenetv1_0.25_96_c3_o2_l5\mobilenetv1_0.25_96_c3_o2_l5_peak_memory\tensor_details.csv
Cleaning up the dataframe.


,id,name,shape,size_b,name_long
0,0,serving_default_input_1:0,"(1, 96, 96, 3)",27648,serving_default_input_1:0
1,61,tfl.quantize,"(1, 96, 96, 3)",27648,tfl.quantize
2,62,mobilenetv1/conv2d/Conv2D,"(1, 48, 48, 8)",18432,mobilenetv1/re_lu/Relu;mobilenetv1/batch_norma...
3,63,mobilenetv1/depthwise_conv2d/BiasAdd,"(1, 48, 48, 8)",18432,mobilenetv1/re_lu_1/Relu;mobilenetv1/batch_nor...
4,64,mobilenetv1/conv2d_1/Conv2D,"(1, 48, 48, 16)",36864,mobilenetv1/re_lu_2/Relu;mobilenetv1/batch_nor...
5,65,mobilenetv1/depthwise_conv2d_1/BiasAdd,"(1, 24, 24, 16)",9216,mobilenetv1/re_lu_3/Relu;mobilenetv1/batch_nor...
6,66,mobilenetv1/conv2d_2/Conv2D,"(1, 24, 24, 32)",18432,mobilenetv1/re_lu_4/Relu;mobilenetv1/batch_nor...
7,67,mobilenetv1/batch_normalization_7/FusedBatchNo...,"(1, 24, 24, 32)",18432,mobilenetv1/re_lu_5/Relu;mobilenetv1/batch_nor...
8,68,mobilenetv1/conv2d_3/Conv2D,"(1, 24, 24, 32)",18432,mobilenetv1/re_lu_6/Relu;mobilenetv1/batch_nor...
9,69,mobilenetv1/depthwise_conv2d_3/BiasAdd,"(1, 12, 12, 32)",4608,mobilenetv1/re_lu_7/Relu;mobilenetv1/batch_nor...


# Layer Analytics from TensorFlow Model

In [46]:
layer_details_df = get_layer_details_df(models_layer_df_path)
layer_details_df

Reading in i:\tinyml\tiny_cnn\models\mobilenetv1_0.25_96_c3_o2_l5\mobilenetv1_0.25_96_c3_o2_l5_layers.csv


,Unnamed: 0,index,layer_name,layer_type,input_shape,input,input_node_1,output_shape,dtype,params,...,input_1,b_i_1,h_i_1,w_i_1,c_i_1,output_1,b_o_1,h_o_1,w_o_1,c_o_1
0,0,0,input_1,<class 'keras.engine.input_layer.InputLayer'>,"(None, 96, 96, 3)","KerasTensor(type_spec=TensorSpec(shape=(None, ...",input_1,"(None, 96, 96, 3)",float32,0,...,"(None, 96, 96, 3)",1,96,96,3,"(None, 96, 96, 3)",1,96,96,3
1,1,1,conv2d,<class 'keras.layers.convolutional.conv2d.Conv...,"(None, 96, 96, 3)","KerasTensor(type_spec=TensorSpec(shape=(None, ...",input_1,"(None, 48, 48, 8)",float32,224,...,"(None, 96, 96, 3)",1,96,96,3,"(None, 48, 48, 8)",1,48,48,8
2,2,2,batch_normalization,<class 'keras.layers.normalization.batch_norma...,"(None, 48, 48, 8)","KerasTensor(type_spec=TensorSpec(shape=(None, ...",conv2d/BiasAdd:0,"(None, 48, 48, 8)",float32,32,...,"(None, 48, 48, 8)",1,48,48,8,"(None, 48, 48, 8)",1,48,48,8
3,3,3,re_lu,<class 'keras.layers.activation.relu.ReLU'>,"(None, 48, 48, 8)","KerasTensor(type_spec=TensorSpec(shape=(None, ...",batch_normalization/FusedBatchNormV3:0,"(None, 48, 48, 8)",float32,0,...,"(None, 48, 48, 8)",1,48,48,8,"(None, 48, 48, 8)",1,48,48,8
4,4,4,depthwise_conv2d,<class 'keras.layers.convolutional.depthwise_c...,"(None, 48, 48, 8)","KerasTensor(type_spec=TensorSpec(shape=(None, ...",re_lu/Relu:0,"(None, 48, 48, 8)",float32,80,...,"(None, 48, 48, 8)",1,48,48,8,"(None, 48, 48, 8)",1,48,48,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82,82,82,global_average_pooling2d,<class 'keras.layers.pooling.global_average_po...,"(None, 3, 3, 256)","KerasTensor(type_spec=TensorSpec(shape=(None, ...",re_lu_26/Relu:0,"(None, 1, 1, 256)",float32,0,...,"(None, 3, 3, 256)",1,3,3,256,"(None, 1, 1, 256)",1,1,1,256
83,83,83,dropout,<class 'keras.layers.regularization.dropout.Dr...,"(None, 1, 1, 256)","KerasTensor(type_spec=TensorSpec(shape=(None, ...",global_average_pooling2d/Mean:0,"(None, 1, 1, 256)",float32,0,...,"(None, 1, 1, 256)",1,1,1,256,"(None, 1, 1, 256)",1,1,1,256
84,84,84,conv2d_14,<class 'keras.layers.convolutional.conv2d.Conv...,"(None, 1, 1, 256)","KerasTensor(type_spec=TensorSpec(shape=(None, ...",dropout/Identity:0,"(None, 1, 1, 2)",float32,514,...,"(None, 1, 1, 256)",1,1,1,256,"(None, 1, 1, 2)",1,1,1,2
85,85,85,reshape,<class 'keras.layers.reshaping.reshape.Reshape'>,"(None, 1, 1, 2)","KerasTensor(type_spec=TensorSpec(shape=(None, ...",conv2d_14/BiasAdd:0,"(None, 2)",float32,0,...,"(None, 1, 1, 2)",1,1,1,2,"(None, 2)",1,1,1,2


# Logging to Weights & Biases

In [47]:
api = wandb.Api()

# Generate run ids
run_id = wandb.util.generate_id()

wandb.init(
    project=PROJECT,
    entity="susbrock",
    id=run_id,
    resume=True)
config = wandb.config
config.id = run_id
config.update(mltk_model_stats)


layers_df_wandb = wandb.Table(dataframe=layers_df)
wandb.log({"tflite_layers_mltk" : layers_df_wandb})
layers_INT8_df_wandb = wandb.Table(dataframe=layers_INT8_df)
wandb.log({"tflite_INt8_layers_mltk" : layers_INT8_df_wandb})

peak_memory_df_wandb = wandb.Table(dataframe=peak_memory_df)
wandb.log({"peak_memory" : peak_memory_df_wandb})

tensor_df_wandb = wandb.Table(dataframe=tensor_df)
wandb.log({"tensor_info" : tensor_df_wandb})

layer_details_df_wandb = wandb.Table(dataframe=layer_details_df)
wandb.log({"tensorflow_layer_details" : layer_details_df_wandb})


wandb.log({"tflite_INt8_layers_plt" : wandb.plot.bar(layers_INT8_df_wandb, "index", "macs", title=model_name)})

wandb.finish()



ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: susbrock. Use `wandb login --relogin` to force relogin
